In [1]:
from vllm import AsyncLLMEngine, SamplingParams
from vllm.engine.arg_utils import AsyncEngineArgs
import json

from pyngrok import ngrok
import nest_asyncio
import asyncio
import uvicorn
from fastapi import FastAPI, Request
from fastapi.responses import StreamingResponse
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

In [2]:
engine_args = AsyncEngineArgs(model="cpm-ai/Ocelot-Ko-self-instruction-10.8B-v1.0", 
                              tensor_parallel_size=8, 
                              max_model_len=4096, 
                              gpu_memory_utilization=0.1)

In [3]:
engine = AsyncLLMEngine.from_engine_args(engine_args)

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-06-13 08:24:49,629	INFO worker.py:1749 -- Started a local Ray instance.


INFO 06-13 08:24:50 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='cpm-ai/Ocelot-Ko-self-instruction-10.8B-v1.0', speculative_config=None, tokenizer='cpm-ai/Ocelot-Ko-self-instruction-10.8B-v1.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=8, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=cpm-ai/Ocelot-Ko-self-instruction-10.8B-v1.0)
INFO 06-13 08:25:17 utils.py:618] Found nccl from library libnccl.so.2
INFO 06-13 08:25:17 pynccl.py:65] vLLM is using nccl==2.20.5
(RayWorkerWrapper pid=2689428) INFO 06-13 08:25:17 utils.py:618] Found nccl from library libnccl.so.2
(RayWorkerWra

In [4]:
sampling_params = SamplingParams(temperature=0, top_p=0.95, top_k=50, max_tokens=4096)

In [5]:
sampling_params_tl = SamplingParams(temperature=0.9, top_p=0.90, top_k=5, max_tokens=4096)

In [6]:
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000", "https://www.cpm-ai.site"],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)



class InputText(BaseModel):
    text: str
    item: str

class TalentText(BaseModel):
    req_id: str
    talent: str
    text: str

# 프론트단 실시간 추론
queue = asyncio.Queue()

@app.get("/sse_01", response_class=StreamingResponse)
async def sse(request: Request):
    headers = {
        "Content-Type": "text/event-stream;charset=utf-8",
        "Cache-Control": "no-cache",
        "Connection": "keep-alive",
    }

    async def event_generator():
        while True:
            data = await queue.get()
            # 줄바꿈 인식 위해 json타입으로
            json_data = json.dumps({"data": data})
            yield f"data: {json_data}\n\n"

    return StreamingResponse(
        event_generator(),
        headers=headers,
        media_type="text/event-stream",
    )

###### 평가 ######

@app.post('/eval')
async def edit_orion(input_text: InputText):
    # 입력값 변수
    item = input_text.item
    item_text = input_text.text

    print('평가 아이템 확인', item)

    # 프롬프트
    prompt = f"""너는 자기소개서 첨삭 전문가야.
주어진 자기소개서를 분석해서 평가해야해.
참고할 사항은 다음과 같아
1. '저는', '제가', '저의'같은 1인칭 주어시첨 표현은 제한적으로 사용.
2. '귀사', '당사', '이 회사' 같은 표현보단 기업의 정식명칭 사용.
3. 문장 구조나 띄어쓰기에 대한 평가는 하지 않음.

출력은 다음의 형식을 따라
[평가]
이런 점이 좋아요:
n. (소제목):
n. (소제목):
...
다시 생각해 봐요:
n. (소제목):
n. (소제목):
...

다음이 자기소개서야 :
[{item_text}]"""
    formatted_prompt = f"instruction: {prompt}\n output:"
    example_input = {
    "prompt": formatted_prompt,
    "stream": False, # assume the non-streaming case
    "request_id": item_text,
    }

    # 비동기 추론
    results_generator = engine.generate(
        example_input["prompt"],
        sampling_params,
        example_input["request_id"])

    final_output = None
    async for request_output in results_generator:
        # front단 실시간 추론을 위한 코드
        await queue.put(request_output.outputs[0].text)
        final_output = request_output
        
    return final_output.outputs[0].text


###### 첨삭 ######

@app.post('/editing')
async def edit_orion(input_text: InputText):
    # 입력값 변수
    item = input_text.item
    item_text = input_text.text

    print('첨삭 아이템 확인', item)
    
    # 프롬프트
    prompt = f"""너는 자기소개서 첨삭 전문가야.
주어진 자기소개서를 첨삭해서 다시 작성해야해.
출력형식은 다음을 지켜야해.

[첨삭]

다음이 자기소개서야 :
[{item_text}]"""
    formatted_prompt = f"instruction: {prompt}\n output:"
    example_input = {
    "prompt": formatted_prompt,
    "stream": False, # assume the non-streaming case
    "request_id": item_text,
    }

    # 비동기 추론
    results_generator = engine.generate(
        example_input["prompt"],
        sampling_params,
        example_input["request_id"])

    final_output = None
    async for request_output in results_generator:
        # front단 실시간 추론을 위한 코드
        await queue.put(request_output.outputs[0].text)
        final_output = request_output
        
    return final_output.outputs[0].text


###### 인재상 첨삭 ######
@app.post('/talent')
async def edit_orion(talent_text: TalentText):
        # 입력값 변수
    talent = talent_text.talent
    user_answer = talent_text.text
    req_id = talent_text.req_id
    

    # 프롬프트
    prompt = f"""
당신은 주어진 인재상을 기준으로 사용자의 자기소개서를 효과적으로 수정하는 역할을 수행합니다. 이 과정을 따라 작업을 진행하세요.

[입력 정보]
인재상: {talent}
자기소개서: {user_answer}

[첨삭 과정]
1. 자기소개서의 각 문단을 신중하게 검토하여 주요 업적과 경험을 식별하고, 이를 인재상의 각 특성과 어떻게 연결할 수 있는지 평가하세요.
2. 인재상을 자세히 읽고, 각 특성이 요구하는 주요 요소들을 이해한 후, 지원자의 경험과 이를 연결하세요.
3. 각 인재상에 부합하는 경험과 성과를 강조할 부분을 식별하고, 인재상과 직접적인 연관이 없는 부분에는 연결 문장을 추가하여 자연스럽게 연계되도록 하세요.
4. 지원자의 능력과 인재상이 어떻게 일치하는지 명확히 보여줄 수 있는 구체적인 예시나 사례를 추가하세요.
5. 자기소개서의 전체적인 흐름을 유지하면서, 자연스러운 문장 구성과 일관된 문체로 인재상을 강조하도록 내용을 통합하세요.
6. 수정된 자기소개서를 처음부터 끝까지 읽어보며 자연스러움과 인재상의 반영 여부를 확인하고, 모든 인재상이 충분히 반영되었는지 검토하세요.

[첨삭]
:

"""
    formatted_prompt = f"instruction: {prompt}\n output:"

    example_input = {
    "prompt": formatted_prompt,
    "stream": False, # assume the non-streaming case
    "request_id": req_id,
    }

    # 비동기 추론
    results_generator = engine.generate(
        example_input["prompt"],
        sampling_params_tl,
        example_input["request_id"]
    )

    final_output = None
    async for request_output in results_generator:
        # front단 실시간 추론을 위한 코드
        await queue.put(request_output.outputs[0].text)
        final_output = request_output
        
    return final_output.outputs[0].text


In [7]:
nest_asyncio.apply()

In [8]:
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8001, ssl_keyfile="/etc/letsencrypt/live/model.cpm-ai.site/privkey.pem", ssl_certfile="/etc/letsencrypt/live/model.cpm-ai.site/fullchain.pem")
    

INFO:     Started server process [2683843]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on https://0.0.0.0:8001 (Press CTRL+C to quit)


INFO:     152.32.251.44:33938 - "GET / HTTP/1.1" 404 Not Found
INFO:     152.32.251.44:42254 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     152.32.251.44:42264 - "GET /robots.txt HTTP/1.1" 404 Not Found
INFO:     152.32.251.44:42274 - "GET /sitemap.xml HTTP/1.1" 404 Not Found
INFO:     87.236.176.224:55489 - "GET / HTTP/1.1" 404 Not Found


INFO:     Shutting down


RuntimeError: Event loop stopped before Future completed.

In [ ]:
# 레거시 코드

In [ ]:
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000", "https://www.cpm-ai.site"],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)



class InputText(BaseModel):
    text: str
    item: str

class TalentText(BaseModel):
    req_id: str
    talent: str
    text: str

# 프론트단 실시간 추론
queue = asyncio.Queue()

@app.get("/sse_01", response_class=StreamingResponse)
async def sse(request: Request):
    headers = {
        "Content-Type": "text/event-stream;charset=utf-8",
        "Cache-Control": "no-cache",
        "Connection": "keep-alive",
    }

    async def event_generator():
        while True:
            data = await queue.get()
            # 줄바꿈 인식 위해 json타입으로
            json_data = json.dumps({"data": data})
            yield f"data: {json_data}\n\n"

    return StreamingResponse(
        event_generator(),
        headers=headers,
        media_type="text/event-stream",
    )

###### 평가 ######

@app.post('/eval')
async def edit_orion(input_text: InputText):
    # 입력값 변수
    item = input_text.item
    item_text = input_text.text

    print('평가 아이템 확인', item)

    # 프롬프트
    prompt = f"""너는 자기소개서 첨삭 전문가야.
주어진 자기소개서를 분석해서 평가해야해.
참고할 사항은 다음과 같아
1. '저는', '제가', '저의'같은 1인칭 주어시첨 표현은 제한적으로 사용.
2. '귀사', '당사', '이 회사' 같은 표현보단 기업의 정식명칭 사용.
3. 문장 구조나 띄어쓰기에 대한 평가는 하지 않음.

출력은 다음의 형식을 따라
[평가]
이런 점이 좋아요:
n. (소제목):
n. (소제목):
...
다시 생각해 봐요:
n. (소제목):
n. (소제목):
...

다음이 자기소개서야 :
[{item_text}]"""
    formatted_prompt = f"instruction: {prompt}\n output:"
    example_input = {
    "prompt": formatted_prompt,
    "stream": False, # assume the non-streaming case
    "request_id": item_text,
    }

    # 비동기 추론
    results_generator = engine.generate(
        example_input["prompt"],
        sampling_params,
        example_input["request_id"])

    final_output = None
    async for request_output in results_generator:
        # front단 실시간 추론을 위한 코드
        await queue.put(request_output.outputs[0].text)
        final_output = request_output
        
    return final_output.outputs[0].text


###### 첨삭 ######

@app.post('/editing')
async def edit_orion(input_text: InputText):
    # 입력값 변수
    item = input_text.item
    item_text = input_text.text

    print('첨삭 아이템 확인', item)
    
    # 프롬프트
    prompt = f"""너는 자기소개서 첨삭 전문가야.
주어진 자기소개서를 첨삭해서 다시 작성해야해.
출력형식은 다음을 지켜야해.

[첨삭]

다음이 자기소개서야 :
[{item_text}]"""
    formatted_prompt = f"instruction: {prompt}\n output:"
    example_input = {
    "prompt": formatted_prompt,
    "stream": False, # assume the non-streaming case
    "request_id": item_text,
    }

    # 비동기 추론
    results_generator = engine.generate(
        example_input["prompt"],
        sampling_params,
        example_input["request_id"])

    final_output = None
    async for request_output in results_generator:
        # front단 실시간 추론을 위한 코드
        await queue.put(request_output.outputs[0].text)
        final_output = request_output
        
    return final_output.outputs[0].text


###### 인재상 첨삭 ######
@app.post('/talent')
async def edit_orion(talent_text: TalentText):
        # 입력값 변수
    talent = talent_text.talent
    user_answer = talent_text.text
    req_id = talent_text.req_id
    

    # 프롬프트
    prompt = f"""당신은 인재상을 기준으로 사용자의 자기소개서를 수정하는 역할을 맡고 있습니다. 아래의 첨삭과정을 지키면서 작업을 수행하세요.

[입력 정보]
인재상: {talent}
자기소개서: {user_answer}

[첨삭 과정]
1. 인재상을 확인하세요.
2. 제공된 자기소개서의 내용과 흐름을 분석하세요.
3. 자기소개서에서 지원자의 인성 및 역량 중 인재상과 부합하는 부분을 식별하세요.
4. 식별된 부분을 강조하고, 해당 문단에서 인재상과 관련된 경험이나 역량을 구체적으로 설명하세요.
5. 모든 문단에 걸쳐 인재상이 고르게 반영되도록 하세요. 
6. 결론은 제외하세요.
7. 출력형식은 다음을 지키세요.

[첨삭]
"""
    formatted_prompt = f"instruction: {prompt}\n output:"

    example_input = {
    "prompt": formatted_prompt,
    "stream": False, # assume the non-streaming case
    "request_id": req_id,
    }

    # 비동기 추론
    results_generator = engine.generate(
        example_input["prompt"],
        sampling_params,
        example_input["request_id"]
    )

    final_output = None
    async for request_output in results_generator:
        # front단 실시간 추론을 위한 코드
        await queue.put(request_output.outputs[0].text)
        final_output = request_output
        
    return final_output.outputs[0].text


In [ ]:

###### 인재상 첨삭 ######
@app.post('/talent')
async def edit_orion(talent_text: TalentText):
        # 입력값 변수
    talent = talent_text.talent
    user_answer = talent_text.text
    req_id = talent_text.req_id
    

    # 프롬프트
    prompt = f"""'{talent}'라는 기업의 인재상을 참고하여 자기소개서를 수정해줘

다음이 자기소개서야 :
[{user_answer}]"""
    formatted_prompt = f"instruction: {prompt}\n output:"

    example_input = {
    "prompt": formatted_prompt,
    "stream": False, # assume the non-streaming case
    "request_id": req_id,
    }

    # 비동기 추론
    results_generator = engine.generate(
        example_input["prompt"],
        sampling_params,
        example_input["request_id"]
    )

    final_output = None
    async for request_output in results_generator:
        # front단 실시간 추론을 위한 코드
        await queue.put(request_output.outputs[0].text)
        final_output = request_output
        
    return final_output.outputs[0].text
